In [1]:
import silence_tensorflow.auto
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import bijectors as tfb
from numpy.random import gamma

from tfprojgamma import ProjectedGamma

# Set random seeds for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# Stickbreak function
def stickbreak(v):
    batch_ndims = len(v.shape) - 1
    cumprod_one_minus_v = tf.math.cumprod(1 - v, axis=-1)
    one_v = tf.pad(v, [[0, 0]] * batch_ndims + [[0, 1]], "CONSTANT", constant_values=1)
    c_one = tf.pad(cumprod_one_minus_v, [[0, 0]] * batch_ndims + [[1, 0]], "CONSTANT", constant_values=1)
    return one_v * c_one

In [ ]:
def create_model(nDat, nCol, nClust, dtype=np.float32):
    return tfd.JointDistributionNamed(dict(
        # Hierarchical mean
        mu = tfd.MultivariateNormalTriL(loc = np.zeros(nCol, dtype) , scale = Lu)
        Lu = tfd.WishartTriL(df = nCol + 10, scale_tril = np.eye(nCol, dtype = dtype) * np.sqrt(nCol + 10))
        
        # Mixture log-shapes
        logalpha = tfd.Independent(
            tfd.MultivariateNormalTriL(loc = mu, scale_tril = Lu),
            reinterpreted_batch_ndims=1
            ),
        
        # Mixture weights (stick-breaking construction)
        alpha = tfd.Gamma(concentration=np.float64(1.0), rate=10.0),
        
        v = lambda alpha: tfd.Independent(
            tfd.Beta(np.ones(nClust - 1, dtype), tf.expand_dims(alpha,-1)),
            reinterpreted_batch_ndims = 1,
            ),
        
        # Observations (likelihood)
        obs = lambda logalpha, v: tfd.Sample(
            tfd.MixtureSameFamily(
                mixture_distribution = tfd.Categorical(probs=stickbreak(v)),
                components_distribution = ProjectedGamma(tf.exp(logalpha)),
                ),
            sample_shape = nDat,
            )
    ))

In [ ]:
ncomponents = 10





In [ ]:
model = create_dp_sb_gmm(nobs=len(simdata['y']), K=ncomponents)
# Define log unnormalized joint posterior density.
def target_log_prob_fn(mu, sigma, alpha, v):
    return model.log_prob(obs=y, mu=mu, sigma=sigma, alpha=alpha, v=v)
